In [3]:
!pwd
%cd drive/MyDrive/nlp100/chapter08

/content
/content/drive/MyDrive/nlp100/chapter08


In [4]:
# knock50

import pandas as pd
from sklearn.model_selection import train_test_split
# FORMAT: ID \t TITLE \t URL \t PUBLISHER \t CATEGORY \t STORY \t HOSTNAME \t TIMESTAMP
df = pd.read_csv("newsCorpora.csv", sep="\t", header=None, names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

# 該当するpublisherの記事を抽出する
publishers = ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']
df = df[df['PUBLISHER'].isin(publishers)]
# TITLEとCATEGORYのみ抽出
df = df[["TITLE", "CATEGORY"]]

#データを分割しシャッフルする
train, test = train_test_split(df, test_size=0.2, shuffle=True)
test, valid = train_test_split(test, test_size=0.5, shuffle=True)

#ファイルに保存する
train.to_csv("train.txt", sep="\t", index=False, header=None)
valid.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)

print("train\n", train["CATEGORY"].value_counts())
print("valid\n", valid["CATEGORY"].value_counts())
print("test\n", test["CATEGORY"].value_counts())

train
 b    4496
e    4220
t    1210
m     746
Name: CATEGORY, dtype: int64
valid
 b    559
e    535
t    158
m     82
Name: CATEGORY, dtype: int64
test
 b    572
e    524
t    156
m     82
Name: CATEGORY, dtype: int64


In [5]:
# knock60

from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [10]:
# knock70
import string
import torch
from tqdm import tqdm

# タイトルを特徴ベクトル化する
def transform_w2v(text):
  # 記号を削除する
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation)) # 記号をスペースに置換する文字列変換を作成
  words = text.translate(table).split() # 変換を実行してスペースで分割する
  vec = []
  # ベクトル化する
  for word in words:
    if word in model:
      vec.append(model[word])
  return torch.tensor(sum(vec) / len(vec))

# 特徴ベクトルを作成
X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])

print(X_train)

# ラベルベクトルを作成
category = {'b': 0, 't': 1, 'e': 2, 'm': 3}
# カテゴリ名を数字に変えてテンソル化
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category[x]).values)

print(y_train)

torch.save(X_train, 'X_train.pt')
torch.save(X_valid, 'X_valid.pt')
torch.save(X_test, 'X_test.pt')
torch.save(y_train, 'y_train.pt')
torch.save(y_valid, 'y_valid.pt')
torch.save(y_test, 'y_test.pt')

tensor([[ 0.0221,  0.1608, -0.0503,  ..., -0.0142,  0.0284,  0.1073],
        [ 0.0812,  0.0148, -0.0837,  ..., -0.0396,  0.0188,  0.0505],
        [-0.0361,  0.0535, -0.0784,  ..., -0.0192,  0.0441, -0.0964],
        ...,
        [-0.0222,  0.1884,  0.0219,  ..., -0.0295,  0.0500, -0.0215],
        [ 0.0419, -0.0294,  0.0578,  ...,  0.0140,  0.0212,  0.0078],
        [ 0.0942, -0.0175, -0.0270,  ..., -0.0232,  0.1205, -0.0556]])
tensor([2, 2, 0,  ..., 2, 2, 2])


In [11]:
# knock71
from torch import nn

class SLPNet(nn.Module):
  def __init__(self, input_size, output_size):
    # 以下の処理を追加
    super().__init__()
    # 全結合層を定義
    self.fc = nn.Linear(input_size, output_size, bias=False)
    # 正規乱数で重みを初期化する
    nn.init.normal_(self.fc.weight, 0.0, 1.0)

  def forward(self, x):
    x = self.fc(x)
    return x

# 単層ニューラルネットワークを初期化
model = SLPNet(300, 4)
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
print(y_hat_1)
Y_hat = torch.softmax(model.forward(X_train[:4]), dim=-1)
print(Y_hat)

tensor([[0.4494, 0.0115, 0.1775, 0.3617]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4494, 0.0115, 0.1775, 0.3617],
        [0.1374, 0.0695, 0.3424, 0.4507],
        [0.2929, 0.0241, 0.0339, 0.6491],
        [0.2620, 0.0510, 0.2106, 0.4764]], grad_fn=<SoftmaxBackward0>)


In [13]:
# knock72

criterion = nn.CrossEntropyLoss()
l_1 = criterion(model(X_train[:1]), y_train[:1])
model.zero_grad()
l_1.backward()
print(f'損失: {l_1}')
print(f'勾配:\n{model.fc.weight.grad}')

l = criterion(model(X_train[:4]), y_train[:4])
model.zero_grad()
l.backward()
print(f'損失: {l}')
print(f'勾配:\n{model.fc.weight.grad}')

損失: 1.728939414024353
勾配:
tensor([[ 0.0099,  0.0723, -0.0226,  ..., -0.0064,  0.0128,  0.0482],
        [ 0.0003,  0.0018, -0.0006,  ..., -0.0002,  0.0003,  0.0012],
        [-0.0182, -0.1322,  0.0414,  ...,  0.0117, -0.0233, -0.0882],
        [ 0.0080,  0.0582, -0.0182,  ..., -0.0052,  0.0103,  0.0388]])
損失: 1.3420171737670898
勾配:
tensor([[ 0.0025,  0.0096,  0.0384,  ..., -0.0146, -0.0183,  0.0496],
        [ 0.0019,  0.0010, -0.0044,  ...,  0.0002,  0.0017, -0.0007],
        [-0.0156, -0.0352,  0.0140,  ...,  0.0136, -0.0045, -0.0365],
        [ 0.0112,  0.0246, -0.0480,  ...,  0.0008,  0.0211, -0.0124]])


In [15]:
# knock73

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# dataset型
class NewsDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return [self.X[idx], self.y[idx]]

# Dataset（ミニバッチ）を作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# Dataloaderを作成
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

model = SLPNet(300, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# 学習
num_epochs = 100
for epoch in tqdm(range(num_epochs)):
  model.train()
  loss_train = 0.0
  for i, (inputs, labels) in enumerate(dataloader_train):
    optimizer.zero_grad()
    # 順伝播
    outputs = model(inputs)
    # 誤差逆伝播
    loss = criterion(outputs, labels)
    loss.backward()
    # 更新
    optimizer.step()

    loss_train += loss.item()
  # 平均損失を計算
  loss_train = loss_train / i
  # 検証データの損失を計算
  model.eval()
  with torch.no_grad():
    inputs, labels = next(iter(dataloader_valid))
    outputs = model(inputs)
    loss_valid = criterion(outputs, labels)
  print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, loss_valid: {loss_valid:.4f}')

  1%|          | 1/100 [00:05<08:51,  5.37s/it]

epoch: 1, loss_train: 0.4767, loss_valid: 0.3416


  2%|▏         | 2/100 [00:09<07:28,  4.57s/it]

epoch: 2, loss_train: 0.3166, loss_valid: 0.3013


  3%|▎         | 3/100 [00:13<06:58,  4.32s/it]

...

epoch: 98, loss_train: 0.2089, loss_valid: 0.2931


 99%|█████████▉| 99/100 [07:17<00:04,  4.32s/it]

epoch: 99, loss_train: 0.2089, loss_valid: 0.2928


100%|██████████| 100/100 [07:21<00:00,  4.41s/it]

epoch: 100, loss_train: 0.2092, loss_valid: 0.2937


In [16]:
# knock74

def calculate_accuracy(model, loader):
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for inputs, labels in loader:
      outputs = model(inputs)
      pred = torch.argmax(outputs, dim=-1)
      total += len(inputs)
      correct += (pred == labels).sum().item()
  return correct / total

acc_train = calculate_accuracy(model, dataloader_train)
acc_test = calculate_accuracy(model, dataloader_test)
print(f'学習データ: {acc_train}')
print(f'評価データ: {acc_test}')

学習データ: 0.9324400299850075
評価データ: 0.8883058470764618
